# Algoritmos e Estruturas de Dados II - U1T1
Aluno: Juscelino Pereira de Araújo

### Introdução

Este notebook implementa a solução de autocompletação de textos usando Árvores AVL, o que constitui o segundo trabalho da primeira unidade da disciplina de Algoritmos e Estruturas de Dados II. A solução foi implementada usando classes de Python previamente implementadas e disponibilizadas pelo professor. Além disso, o corpus utilizado foi uma tradução em português da Bíblia disponibilizada no sítio https://umsocorpo.com.br/downloads/biblia-sagrada-em-txt-versao-revista-e-corrigida/. A seguir, são descritos os passos seguidos para a construção da solução.

### Pré-Processamento



Inicialmente o arquivo txt utilizado é carregado na forma de String. Nesse caso, obtemos inicialmente a String `biblia_string`.

In [ ]:
biblia_arquivo = open("biblia-em-txt.txt", "r")
biblia_string = biblia_arquivo.read()
biblia_arquivo.close()


FileNotFoundError: [Errno 2] No such file or directory: 'biblia-em-txt.txt'

Em seguida, são removidos caracteres especiais, incluindo números e sinais de pontuação.

In [ ]:
caracteres_removiveis = "0123456789,.;?[]{}'!@#$%&*()_^:"
for c in caracteres_removiveis:
  biblia_string = biblia_string.replace(c, "")

O passo seguinte é a conversão de todas as letras para minúsculas.

In [ ]:
biblia_string = biblia_string.lower()

Além disso, letras (no caso do português somente as vogais) com acentuação tem seus acentos removidos.

In [ ]:
a_com_acento = "áàãâ"
e_com_acento = "éê"
o_com_acento = "óõô"
for l in a_com_acento:
  biblia_string = biblia_string.replace(l, "a")
for l in e_com_acento:
  biblia_string = biblia_string.replace(l, "e")
for l in o_com_acento:
  biblia_string = biblia_string.replace(l, "o")
biblia_string = biblia_string.replace("í", "i")
biblia_string = biblia_string.replace("ú", "u")
biblia_string = biblia_string.replace("ç", "c")

Removidos os caracteres especiais e trocadas as letras com acentos, a string é dividida em palavras. A separação acontece por meio de espaços. Por isso, as quebras de linhas e os hífens são inicialmente convertidas em espaços para que a separação ocorra normalmente. Com isso, é criado o arquivo `biblia_lista`, que consiste em uma lista com as palavras obtidas da separação realizada.

In [ ]:
biblia_string = biblia_string.replace("\n", " ")
biblia_string = biblia_string.replace("-", " ")
biblia_lista = biblia_string.split(" ")


Como seria de se esperar um texto tão grande, muitas palavras são repetidas. O código executado a seguir remove as palavras repetidas. Isso é feito gerando uma estrutura de conjunto com os elementos da lista e depois gerando uma nova lista a partir desse conjunto. Convém lembrar que isso advém do fato de que em estruturas de tipo conjunto não há multiplicidade de elementos.

In [ ]:
biblia_lista = list(set(biblia_lista))

Algumas das palavras não são de interesse, como artigos, preposições e algumas conjunções. Elas são retiradas da lista.

In [ ]:
palavras_removiveis = [" ", "o", "os", "a", "as", "um", "uma", "de", "do", "dos", "da", "das", "dum", "duma", "em", "no", "nos", "na", "nas", "para", "que", "mas", "e", "se"]
for p in palavras_removiveis:
  if p in biblia_lista:
    biblia_lista.remove(p)

A lista de palavras é então ordenada, sendo armazenada na variável `biblia_lista_ordenada`:

In [ ]:
biblia_lista_ordenada = sorted(biblia_lista)

Já temos uma lista de palavras, o que funcionará como estrutura linear de busca para efeitos de comparção. Quanto à estrutura em árvore, na seção a seguir são descritas as classes previamente criadas e que seriam utilizadas aqui.

### Árvores AVL

A seguir estão as classes já desenvolvidas em Python que serão empregadas mais adiante. Mais detalhes poem ser encontrados no repositório do Professor Ivanovitch Silva, disponível [aqui](https://github.com/ivanovitchm/datastructure/blob/main/lessons/week_02/avl.ipynb).

- `Node` (Nó)
- `BST` (Árvore Binária de Busca)
- `AVLNode` (Nó da Árvore AVL)
- `AVLTree` (Árvore AVL)

Node:


In [ ]:
class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None

BST:

In [ ]:
class BST:
    """
    Represents a Binary Search Tree (BST).

    Attributes:
        root (Node or None): The root node of the tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """Initializes an empty BST."""
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the BST.

        Args:
            value: The value to be added to the tree.

        If the tree is empty, the value becomes the root. Otherwise, the method uses
        a recursive helper function to find the appropriate position to maintain the BST property.
        """
        if self.root is None:
            self.root = Node(value)
        else:
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the BST.

        Args:
            current_node (Node): The node to start the search for the insert position from.
            value: The value to be added to the BST.

        The method determines if the new value should be placed to the left or right of
        the current node. If the target position is empty, the value is inserted.
        Otherwise, the function calls itself recursively with the respective child node.
        """
        if value <= current_node.value:
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):
        """
        Recursively checks if the BST contains the specified value starting from a given node.

        Args:
            current_node (Node): The node to start the search from.
            value: The value to search for in the BST.

        Returns:
            bool: True if the value exists in the subtree rooted at current_node, otherwise False.
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks if the BST contains the specified value.

        Args:
            value: The value to search for in the BST.

        Returns:
            bool: True if the BST contains the value, otherwise False.
        """
        return self._contains(self.root, value)


AVLNode:

In [ ]:
class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        height (int): The height of the subtree rooted at this node,
                      initializes to 1 when the node is created.
        imbalance (int): The imbalance factor of this node, calculated
                         as the difference between the heights of the left
                         and right subtrees. Initializes to 0.

    Inherits from:
        Node: Inherits attributes and methods from the Node class.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in this node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height


AVLTree:

In [ ]:
class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def add(self, value):
        """
        Overrides the add method in the BST class to handle AVL Tree balancing.
        """
        self.root = self._add_recursive(self.root, value)  # Note that self.root is updated here


    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        This method also ensures the tree remains balanced by updating node heights and performing rotations as needed.

        Args:
            current_node (AVLNode or Node or None): The node from which to start the search for the insert position.
            value (Any): The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that either gets inserted or the node that was already present at that position.

        Notes:
            1. The method first checks if the `current_node` is an instance of the base class `Node`.
              If it is, the method casts it to `AVLNode` to ensure AVL properties are maintained. This is especially
              useful if the first node added to the tree is of type `Node`; this ensures subsequent nodes will be of
              type `AVLNode`.
            2. The method also balances the tree by calling the `_balance` method if the imbalance factor
              of a node reaches 2 or -2 after an insert operation.
        """

        # If the current node is None, return a new AVLNode containing the value
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary to not change the add() in the BST class.
        # When the first node is added, the type of the root is Node, so we need to cast it
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
            current_node = AVLNode(current_node.value)
            current_node.left_child = self.root.left_child
            current_node.right_child = self.root.right_child
            self.root = current_node

        # Determine whether the value should be inserted to the left or right subtree
        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        # Check if tree balancing is needed and balance if necessary
        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        A left rotation is used to balance an AVL Tree when the right subtree of a node
        becomes higher than the left subtree. The method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        A right rotation is used to balance an AVL Tree when the left subtree of a node
        becomes higher than the right subtree. This method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node around which the rotation will be performed.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the left subtree of 'node')
        pivot = node.left_child

        # Update the left child of 'node' to be the right child of the pivot
        node.left_child = pivot.right_child

        # Set the right child of the pivot to be the node
        pivot.right_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot

    def _balance(self, node):
      """
      Balances the subtree rooted at the given node by performing rotations as needed.

      If the imbalance factor of the given node is 2 or -2, rotations are performed
      to bring the subtree back into balance. This method also takes into account
      the imbalance factors of the child nodes to decide which type of rotation is needed
      (single or double).

      Args:
          node (AVLNode): The root node of the subtree that needs to be balanced.

      Returns:
          AVLNode: The new root node of the balanced subtree.

      Note:
          This method assumes that the height and imbalance factor of each node are up-to-date.
      """

      # Case 1: Left subtree is higher than right subtree
      if node.imbalance == 2:
          pivot = node.left_child
          # Single right rotation
          if pivot.imbalance == 1:
              return self._rotate_right(node)
          # Double rotation: Left-Right
          else:
              node.left_child = self._rotate_left(pivot)
              return self._rotate_right(node)
      # Case 2: Right subtree is higher than left subtree
      else:
          pivot = node.right_child
          # Single left rotation
          if pivot.imbalance == -1:
              return self._rotate_left(node)
          # Double rotation: Right-Left
          else:
              node.right_child = self._rotate_right(pivot)
              return self._rotate_left(node)



### Função de Autocompletamento de Palavras

Para realizar o autocompletamento das palavras

In [ ]:
def autocomplete_lista_ordenada(lista, palavra):
  saida = []
  for p in lista:
    if p[0:len(palavra)] < palavra:
      continue
    elif p[0:len(palavra)] == palavra:
      saida.append(p)
    else:
      break;
  return saida

In [ ]:
import time
palavra = input("Prefixo a pesquisar: ")
t_inicial_lista = time.time_ns()
for p in autocomplete_lista_ordenada(biblia_lista_ordenada, palavra):
  print(p)
t_final_lista = time.time_ns()
print("Usando lista ordenada: tempo de Execução de ",t_final_lista - t_inicial_lista, " nanossegundos.")

Prefixo a pesquisar: no
noa
noadias
noba
nobai
nobe
nobre
nobres
nocivas
nocivos
nodabe
node
nodoas
noe
noemi
nofa
nofegue
noga
nogueiras
noite
noites
noiva
noivo
nojenta
nojo
nomades
nome
nomeada
nomeado
nomeados
nomeia
nomeie
nomes
nominalmente
nona
nono
nora
noras
nordeste
norma
norte
nossa
nossas
nosso
nossos
nota
notai
notar
notaram
notaras
notaste
notaveis
notavel
noteis
noticia
noticias
noticieis
notificado
notificamos
notificando
notificar
notificou
notoria
notorio
notorios
notou
noturna
noturnas
noturno
noturnos
noutra
noutras
noutro
nova
novamente
novas
nove
novecentos
noventa
novidade
novidades
novilha
novilho
novilhos
novo
novos
nozes
Usando lista ordenada: tempo de Execução de  31080496  nanossegundos.


In [ ]:
def autocomplete_no(no, palavra):
  saida = []
  if no != None:
    inicio = no.value[0:len(palavra)]
    if inicio >= palavra:
      saida += autocomplete_no(no.left_child, palavra)
    if inicio == palavra:
      saida.append(no.value)
    if inicio <= palavra:
      saida += autocomplete_no(no.right_child, palavra)
  return saida

def autocomplete_arvore_AVL(arvore, palavra):
  return sorted(autocomplete_no(arvore.root, palavra))

In [ ]:
biblia_arvore_AVL = AVLTree();
for p in biblia_lista_ordenada:
  biblia_arvore_AVL.add(p)
t_inicial_arvore = time.time_ns()
for p in autocomplete_arvore_AVL(biblia_arvore_AVL, palavra):
  print(p)
t_final_arvore = time.time_ns()
print("Usando árvore AVL: tempo de Execução de ",t_final_arvore - t_inicial_arvore, " nanossegundos.")

noa
noadias
noba
nobai
nobe
nobre
nobres
nocivas
nocivos
nodabe
node
nodoas
noe
noemi
nofa
nofegue
noga
nogueiras
noite
noites
noiva
noivo
nojenta
nojo
nomades
nome
nomeada
nomeado
nomeados
nomeia
nomeie
nomes
nominalmente
nona
nono
nora
noras
nordeste
norma
norte
nossa
nossas
nosso
nossos
nota
notai
notar
notaram
notaras
notaste
notaveis
notavel
noteis
noticia
noticias
noticieis
notificado
notificamos
notificando
notificar
notificou
notoria
notorio
notorios
notou
noturna
noturnas
noturno
noturnos
noutra
noutras
noutro
nova
novamente
novas
nove
novecentos
noventa
novidade
novidades
novilha
novilho
novilhos
novo
novos
nozes
Usando árvore AVL: tempo de Execução de  26329586  nanossegundos.
